# LSTM을 이용하여 텍스트 생성하기

#### 1. 데이터에 대한 이해와 전처리

1-1. 라이브러리 불러오기

In [1]:
import pandas as pd 
from string import punctuation 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import numpy as np 
from tensorflow.keras.utils import to_categorical 

C:\Users\isfs0\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\isfs0\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\isfs0\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### 라이브러리의 기능
1. from string import punctuation
 - string 모듈에서 punctuation 상수를 가져옵니다.
 - punctuation 상수에는 일반적으로 텍스트에서 제거해야 할 구두점 문자가 포함이러한 구두점 문자는 일반적으로 텍스트 전처리나 텍스트 처리 작업 중에서 제거되거나 다루어집니다. 예를 들어, 쉼표, 마침표, 느낌표, 물음표 등이 포함될 수 있습니다.
 
2. from tensorflow.keras.utils import to_categorical
 - 다중 클래스 분류 작업을 위해 사용, 정수로 된 클래스 레이블을 이진 또는 원-핫 인코딩된 벡터로 변환하는 데 사용
 - 변환된 벡터는 신경망 모델의 출력 레이어에서 사용되며, 각 클래스에 대한 확률 값을 반환
 
3. 짚고넘어가기 - 원핫 인코딩
 - 주형 데이터(카테고리, 클래스, 레이블)를 수치형 데이터로 변환하는 방법 중 하나

#### 2. 모델 설계하기

In [2]:
df=pd.read_csv('ArticlesApril2018.csv') # 데이터  로드 
df.head() 

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [3]:
print('열의  개수: ',len(df.columns)) 
print(df.columns)

열의  개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [4]:
df['headline'].isnull().values.any() #null이 하나도 없으면 false

False

In [5]:
headline = [] # 리스트  선언
headline.extend(list(df.headline.values)) # 헤드라인의  값들을  리스트로  저장 
headline[:5] # 상위  5개만  출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [6]:
print('총  샘플의  개수  : {}'.format(len(headline))) # 현재  샘플의  개수

# {}는 왜 썼을까?
# .format의 의미는 무엇일까?

총  샘플의  개수  : 1324


In [7]:
headline = [n for n in headline if n != "Unknown"] 
# Unknown(잡음) 값을  가진  샘플  제거 
print('노이즈값  제거  후  샘플의  개수  : {}'.format(len(headline))) 
# 제거  후  샘플의  개수

노이즈값  제거  후  샘플의  개수  : 1214


In [8]:
headline[:5] # 5개의  샘플  출력  확인

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [9]:
def repreprocessing(s): 
    # repreprocessing 함수 정의
    # s: 입력으로 받는 문자열

    # 1. 비 ASCII 문자를 제거하고 ASCII 문자로 변환
    s = s.encode("utf8").decode("ascii", 'ignore')

    # 2. 구두점을 제거하고 동시에 소문자화
    # ''.join(c for c in s if c not in punctuation)은 
    # s에서 구두점을 제외한 문자들을 합친다.
    # 그리고 이어지는 .lower()로 소문자로 변환한다.
    return ''.join(c for c in s if c not in punctuation).lower() 

# repreprocessing 함수는 이제 정의되었습니다.
# 이 함수는 텍스트를 전처리하여 비 ASCII 문자를 제거하고 구두점을 제거한 후 소문자로 변환하는 역할을 합니다.

# 아래는 repreprocessing 함수를 headline 리스트의 각 요소에 적용하여 전처리된 텍스트를 생성하는 부분입니다.
text = [repreprocessing(x) for x in headline]

# text 리스트는 전처리된 텍스트로 채워집니다.

# 이제, text 리스트의 처음 5개 요소를 출력합니다.
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [10]:
# Tokenizer 객체 생성
t = Tokenizer()

# Tokenizer에 텍스트 데이터를 학습시킴
t.fit_on_texts(text)

# 단어 집합의 크기 계산
vocab_size = len(t.word_index) + 1

# 단어 집합의 크기를 출력
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [11]:
# 학습할 데이터 생성
sequences = list() 

for line in text: 
    encoded = t.texts_to_sequences([line])[0] # 각  샘플에  대한  정수  인코딩 
    for i in range(1, len(encoded)): 
        sequence = encoded[:i+1] 
        sequences.append(sequence) 

sequences[:11] # 11개의  샘플  출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [12]:
len(sequences)

7803

In [13]:
index_to_word={} 
for key, value in t.word_index.items(): # 인덱스를  단어로  바꾸기  위해  index_to_word를  생성 
    index_to_word[value] = key  

print('빈도수  상위  582번  단어  : {}'.format(index_to_word[582]))

빈도수  상위  582번  단어  : offer


In [14]:
max_len=max(len(l) for l in sequences) # 가장  긴  샘플의  길이  확인 
print('샘플의  최대  길이  : {}'.format(max_len))

샘플의  최대  길이  : 24


In [15]:
# 가장  긴  샘플의  길이인  24로  모든  샘플의  길이를  패딩 
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') 
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [16]:
sequences = np.array(sequences) 
X = sequences[:,:-1] # 학습 데이터 
y = sequences[:,-1]  # 정답 데이터 
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [17]:
print(y[:3]) # 레이블  3개  출력

[ 269  371 1115]


In [18]:
y = to_categorical(y, num_classes=vocab_size) # 레이블  데이터  y에  대해서  원-핪  인코딩  수행 
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 2. 모델  설계하기

In [19]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, Dense, LSTM 

In [20]:
model = Sequential() 
# y데이터를  분리하였으므로  이제  X데이터의  길이는  기존  데이터의  길이  - 1 
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 10: 계산된 결과를 몇개까지 만들것인다
model.add(LSTM(128)) 
model.add(Dense(vocab_size, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
244/244 - 61s - loss: 7.6463 - accuracy: 0.0279 - 61s/epoch - 251ms/step
Epoch 2/100
244/244 - 31s - loss: 7.1225 - accuracy: 0.0310 - 31s/epoch - 126ms/step
Epoch 3/100
244/244 - 29s - loss: 6.9868 - accuracy: 0.0286 - 29s/epoch - 118ms/step
Epoch 4/100
244/244 - 35s - loss: 6.8691 - accuracy: 0.0374 - 35s/epoch - 144ms/step
Epoch 5/100
244/244 - 26s - loss: 6.7344 - accuracy: 0.0442 - 26s/epoch - 107ms/step
Epoch 6/100
244/244 - 33s - loss: 6.5735 - accuracy: 0.0434 - 33s/epoch - 135ms/step
Epoch 7/100
244/244 - 34s - loss: 6.4197 - accuracy: 0.0499 - 34s/epoch - 139ms/step
Epoch 8/100
244/244 - 36s - loss: 6.1873 - accuracy: 0.0559 - 36s/epoch - 146ms/step
Epoch 9/100
244/244 - 35s - loss: 5.9888 - accuracy: 0.0595 - 35s/epoch - 145ms/step
Epoch 10/100
244/244 - 37s - loss: 5.8008 - accuracy: 0.0627 - 37s/epoch - 150ms/step
Epoch 11/100
244/244 - 35s - loss: 5.6239 - accuracy: 0.0693 - 35s/epoch - 143ms/step
Epoch 12/100
244/244 - 32s - loss: 5.4553 - accuracy: 0.0730 - 

Epoch 99/100
244/244 - 9s - loss: 0.6445 - accuracy: 0.8709 - 9s/epoch - 36ms/step
Epoch 100/100
244/244 - 9s - loss: 0.6320 - accuracy: 0.8733 - 9s/epoch - 37ms/step


In [21]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재  단어, 반복핛  횟수 
    init_word = current_word # 처음  들어온  단어도  마지막에  같이  출력하기위해  저장 
    sentence = '' 
    for _ in range(n): # n번  반복 
        encoded = t.texts_to_sequences([current_word])[0] # 현재  단어에  대핚  정수  인코딩 
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에  대한  패딩 
        result = model.predict_classes(encoded, verbose=0) 
        for word, index in t.word_index.items(): 
            if index == result: # 맊약  예측핚  단어와  인덱스와  동일한  단어가  있다면 
                break # 해당  단어가  예측  단어이므로  break 
        current_word = current_word + ' '  + word # 현재  단어  + ' ' + 예측  단어를  현재  단어로  변경 
        sentence = sentence + ' ' + word # 예측  단어를  문장에  저장 
        
    sentence = init_word + sentence 
    return sentence

In [22]:
# 임의의  단어  'i'에  대해서  10개의  단어를  추가  생성 
print(sentence_generation(model, t, 'i', 10))

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
# 임의의  단어  'how'에  대해서  10개의  단어를  추가  생성 
print(sentence_generation(model, t, 'how', 10))